    # conda install - c conda-forge textblob
    # conda install - c conda-forge tweepy
    # conda install - c conda-forge numpy

In [ ]:
import tweepy
from textblob import TextBlob as tb
import numpy as np
import json
import pymongo

In [ ]:
api_key = ''
api_key_secret = ''
access_token = ''
access_token_secret = ''

In [ ]:
# autenticação
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
'''
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)
'''

In [ ]:
#definir uma palavra que deseja pesquisar no twitter
keyword = input('Informe a keyword')

In [ ]:
#fazer a busca por palavra chave
#tweets = api.search(q=keyword, count=100, result_type='mixed')
tweets = api.search(q=keyword, count=1000, result_type='mixed', tweet_mode='extended', lang='pt')

In [ ]:
#total de tweets coletados
print("Total de Tweet(s) coletados %s." %  (len(tweets)))

In [ ]:
#impressao do nome do usuario e texto
for tweet in tweets:
    print("Usuário: %s" % {tweet.user.screen_name})
    #print(" Texto: %s" % {tweet.text})
    print(" Texto: %s" % {tweet.full_text}) #tweet_mode='extended'

In [ ]:
#analise de polaridade 
#vamos usar a função sentiment_polarity do pacote textblob
#retorna o valor -1 a 1 sendo que quanto mais proximo do 1 menos chateado a pessoa esta

#variavel que vai armazenar as polaridades
analysis = None
#lista vazia para armazenar score
tweet_score = []

In [ ]:
#aqui estou executando a analise de polaridade
for tweet in tweets:
    print('**', tweet.full_text)
    analysis = tb(tweet.full_text)
    polarity = analysis.sentiment.polarity
    tweet_score.append(polarity)

In [ ]:
#vetor de polaridade
print('Vetor de polaridade', tweet_score)

In [ ]:
#média de sentimentos
print('Média de Sentimentos : ' + str(np.mean(tweet_score)))

In [ ]:
#ajustar o textblob(polaritics) para o idioma ingles ( que não foram postados em inglês )
polarities_score = []

for tweet in tweets:
    analysis = tb(tweet.full_text)
    
    if analysis.detect_language != 'en':
        traducao = tb(str(analysis.translate(to='en')))
        print('Texto Traduzido %s' % traducao)
        polarity = traducao.sentiment.polarity
    else:
        polarity = analysis.sentiment.polarity
        
    polarities_score.append(polarity)
    


In [ ]:
#vetor de polaridade
print('Vetor de polaridade', polarities_score)

In [ ]:
#média de sentimentos
print('Média de Sentimentos : ' + str(np.mean(polarities_score)))

In [ ]:
#armazenar a coleta em arquivo formato json
local = 'c:\igti'

with open(local+'\\tweet_'+keyword+'.json', 'a', encoding='utf8') as filename:
    
    for tweet in tweets:
        status = tweet
        #converte para string
        json_str = json.dumps(status._json)
        #deserealiza a string para um objeto python dict
        json_dict = json.loads(json_str)
        #grava o tweet deserealizado no arquivo
        json.dump(json_dict, filename, ensure_ascii=False, sort_keys=True, indent=4, separators=(',',':'))


In [ ]:
#armazenar o mongoDB
conexao = pymongo.MongoClient('localhost',27017)

#banco de dados
db = conexao.bdtweets

In [ ]:
db.tweets_keywords.drop()

In [ ]:
#seleciona a coleção
collections = db.tweets_keywords

In [ ]:
#lista a quantidade de tweets na colecao
db.tweets_keywords.count_documents({})

In [ ]:
i = 0
for tweet in tweets:
    db.tweets_keywords.insert_one(tweet._json)
    i+= 1    

In [ ]:
print('Quantidade de Tweets importados %s' % i)

In [ ]:
db.tweets_keywords.find_one()